In [2]:
import random

# ==== RATING CALCULATION ====

def compute_rating(wins, losses, runs_scored, runs_allowed):
    games = wins + losses
    win_pct = wins / games
    run_diff_per_game = (runs_scored - runs_allowed) / games
    run_diff_norm = (run_diff_per_game + 3) / 6  # Normalize to [0, 1]
    pythag = (runs_scored ^ 2) / ((runs_scored ^ 2) + (runs_allowed ^ 2))
    return (0.25 * win_pct) + (0.25 * run_diff_norm) + (5 * pythag)


# ==== GAME & SERIES SIMULATION ====

def game_win_prob(team_rating, opp_rating, home_field_advantage=0):
    return (team_rating + home_field_advantage) / (team_rating + home_field_advantage + opp_rating)

def simulate_game(team_rating, opp_rating, is_home, hfa=0.05):
    prob = game_win_prob(team_rating, opp_rating, hfa if is_home else 0)
    return random.random() < prob

def get_home_games(round_name, home_team_is_A):
    formats = {
        "wildcard":       [True, True, True],
        "divisional":     [True, True, False, True, False],
        "lcs":            [True, True, False, False, False, True, True],
        "world_series":   [True, True, False, False, False, True, True],
    }
    base = formats[round_name]
    return base if home_team_is_A else [not g for g in base]

def simulate_series(team_A, team_B, round_name):
    series_format = {
        "wildcard":     (2, 3),
        "divisional":   (3, 5),
        "lcs":          (4, 7),
        "world_series": (4, 7)
    }[round_name]

    wins_needed, max_games = series_format

    # HOME FIELD LOGIC
    if round_name == "world_series":
        if team_A["wins"] != team_B["wins"]:
            home_team_is_A = team_A["wins"] > team_B["wins"]
        else:
            home_team_is_A = team_A["seed"] < team_B["seed"]
    else:
        home_team_is_A = team_A["seed"] < team_B["seed"]

    home_games = get_home_games(round_name, home_team_is_A)
    home_team = team_A if home_team_is_A else team_B

    team_A_wins = 0
    team_B_wins = 0

    print(f"\n🎯 {round_name.upper()} SERIES: {team_A['name']} (Seed {team_A['seed']}) vs {team_B['name']} (Seed {team_B['seed']})")
    print(f"🏟 Home field advantage: {home_team['name']}")

    for i in range(max_games):
        is_home_for_A = home_games[i]
        winner = simulate_game(team_A['rating'], team_B['rating'], is_home_for_A)

        if winner:
            team_A_wins += 1
            print(f"Game {i+1} ({'Home' if is_home_for_A else 'Away'} for {team_A['name']}): {team_A['name']} win")
        else:
            team_B_wins += 1
            print(f"Game {i+1} ({'Home' if not is_home_for_A else 'Away'} for {team_B['name']}): {team_B['name']} win")

        if team_A_wins == wins_needed:
            print(f"🏆 {team_A['name']} win the {round_name} series {team_A_wins}-{team_B_wins}")
            return team_A
        elif team_B_wins == wins_needed:
            print(f"🏆 {team_B['name']} win the {round_name} series {team_B_wins}-{team_A_wins}")
            return team_B


# ==== POSTSEASON BRACKET SIMULATION ====

def simulate_postseason(teams_AL, teams_NL):
    print("\n⚾ SIMULATING MLB POSTSEASON BRACKET\n")

    # Wild Card Round
    ALWC1 = simulate_series(teams_AL[5], teams_AL[4], "wildcard")
    ALWC2 = simulate_series(teams_AL[6], teams_AL[3], "wildcard")
    NLWC1 = simulate_series(teams_NL[5], teams_NL[4], "wildcard")
    NLWC2 = simulate_series(teams_NL[6], teams_NL[3], "wildcard")

    # Division Series
    ALDS1 = simulate_series(teams_AL[1], ALWC1, "divisional")
    ALDS2 = simulate_series(teams_AL[2], ALWC2, "divisional")
    NLDS1 = simulate_series(teams_NL[1], NLWC1, "divisional")
    NLDS2 = simulate_series(teams_NL[2], NLWC2, "divisional")

    # League Championship
    ALCS = simulate_series(ALDS1, ALDS2, "lcs")
    NLCS = simulate_series(NLDS1, NLDS2, "lcs")

    # World Series
    WS_winner = simulate_series(ALCS, NLCS, "world_series")
    print(f"\n🏁 WORLD SERIES CHAMPION: {WS_winner['name']} 🏆\n")


# ==== TEAM DATA ====

def create_team(name, wins, losses, runs_scored, runs_allowed, seed):
    return {
        "name": name,
        "wins": wins,
        "losses": losses,
        "runs_scored": runs_scored,
        "runs_allowed": runs_allowed,
        "seed": seed,
        "rating": compute_rating(wins, losses, runs_scored, runs_allowed)
    }

teams_AL = [
    None,
    create_team("Detroit Tigers", 59, 35, 468, 360, 1),
    create_team("Houston Astros", 55, 38, 409, 346, 2),
    create_team("Toronto Blue Jays", 54, 39, 427, 407, 3),
    create_team("New York Yankees", 52, 41, 487, 381, 4),
    create_team("Tampa Bay Rays", 50, 44, 444, 378, 5),
    create_team("Boston Red Sox", 50, 45, 481, 425, 6),
]

teams_NL = [
    None,
    create_team("Los Angeles Dodgers", 56, 38, 504, 438, 1),
    create_team("Chicago Cubs", 55, 38, 503, 379, 2),
    create_team("Philadelphia Phillies", 54, 39, 430, 365, 3),
    create_team("Milwaukee Brewers", 53, 40, 435, 367, 4),
    create_team("New York Mets", 53, 41, 413, 374, 5),
    create_team("San Francisco Giants", 51, 43, 388, 368, 6),
]

# ==== RUN THE SIMULATION ====

simulate_postseason(teams_AL, teams_NL)


⚾ SIMULATING MLB POSTSEASON BRACKET


🎯 WILDCARD SERIES: Tampa Bay Rays (Seed 5) vs New York Yankees (Seed 4)
🏟 Home field advantage: New York Yankees
Game 1 (Home for New York Yankees): New York Yankees win
Game 2 (Away for Tampa Bay Rays): Tampa Bay Rays win
Game 3 (Home for New York Yankees): New York Yankees win
🏆 New York Yankees win the wildcard series 2-1

🎯 WILDCARD SERIES: Boston Red Sox (Seed 6) vs Toronto Blue Jays (Seed 3)
🏟 Home field advantage: Toronto Blue Jays
Game 1 (Home for Toronto Blue Jays): Toronto Blue Jays win
Game 2 (Away for Boston Red Sox): Boston Red Sox win
Game 3 (Away for Boston Red Sox): Boston Red Sox win
🏆 Boston Red Sox win the wildcard series 2-1

🎯 WILDCARD SERIES: New York Mets (Seed 5) vs Milwaukee Brewers (Seed 4)
🏟 Home field advantage: Milwaukee Brewers
Game 1 (Home for Milwaukee Brewers): Milwaukee Brewers win
Game 2 (Home for Milwaukee Brewers): Milwaukee Brewers win
🏆 Milwaukee Brewers win the wildcard series 2-0

🎯 WILDCARD SERIES: San Fra